## Testing the ngram model

**We support:**
- n_grams of different length
- word vectors are weighted with tf-idf

**We need:**
- To be sure of the columns classes we want (Atm: `FIRTNAME, NAME, ADDRESS, CITY, DATE, CODE, ID, STRING`)
- A way to reduce the number of feature (maybe total number of occurence per feature)
- Scale to a column-like vision to enhance the distinction between STRING/FIRSTNAME/NAME/CODE to use redundancy: use number of different items
- Add more features: number of words, ending pattern (with voyelle), ratio voyelle/consonne
- In this vision, use the column name to confirm or enhance classification.


In [1]:
import loader
import ngram
import numpy as np

In [2]:
source = [
    ('firstname', 'firstnames.firstname', 3),
    ('name', 'names.name', 3),
    ('code', 'patients.gender', 3),
    ('code', 'admissions.marital_status', 3),
    ('code', 'admissions.religion', 3),
    ('code', 'admissions.insurance', 3),
    ('code', 'admissions.admission_location', 3),
    ('code', 'prescriptions.drug_type', 3),
    ('date', 'prescriptions.startdate', 3),
    ('date', 'admissions.admittime', 3),
    ('id', 'admissions.hadm_id', 3),
    ('id', 'admissions.subject_id', 3),
    ('address', 'addresses.road', 3),
    ('city', 'addresses.city', 3)
]

In [3]:
dataset = []
labels = {}
for column in source:
    dataset.append(column[1:])
    labels[column[1]] = column[0].upper()

In [4]:
columns = loader.fetch_columns(dataset, limit=10000)

In [5]:
clf = ngram.NGramClassifier()
X_train, y_train, X_test, y_test = clf.preprocess(columns, labels)

In [6]:
# X_train, y_train, X_test, y_test

In [7]:
clf.fit(X_train, y_train, ngram_range=(2, 3))
y_pred = clf.predict(X_test)
clf.score(y_pred, y_test)

CITY                	25/28	   89.29% 	(FP:0)
DATE                	30/30	   100.0% 	(FP:0)
ID                  	35/35	   100.0% 	(FP:0)
CODE                	18/25	   72.0% 	(FP:3)
NAME                	20/26	   76.92% 	(FP:20)
FIRSTNAME           	10/24	   41.67% 	(FP:6)
ADDRESS             	32/32	   100.0% 	(FP:1)
SCORE 170/200 :   85.0%
